In [168]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
import numpy as np
import pandas as pd
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , auc
from sklearn.metrics.cluster import fowlkes_mallows_score as gmean
import statistics as stat

In [169]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [170]:
df=pd.read_csv('/content/drive/MyDrive/glass-0-1-6_vs_2.csv')
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,negative
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,negative
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,negative
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,negative
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,negative
...,...,...,...,...,...,...,...,...,...,...
187,1.518326,13.33015,3.34056,1.53869,72.1396,0.55890,8.99156,0.0,0.0000,positive
188,1.516708,13.23705,3.56955,1.38140,72.6996,0.55890,8.44280,0.0,0.0510,positive
189,1.516549,13.40995,3.38995,1.27868,72.6380,0.52164,8.64724,0.0,0.0000,positive
190,1.521264,14.32100,3.90181,0.82928,71.5012,0.00000,9.48652,0.0,0.0000,positive


In [171]:
df["Class"].value_counts()
class_label=df['Class']
class_label

0      negative
1      negative
2      negative
3      negative
4      negative
         ...   
187    positive
188    positive
189    positive
190    positive
191    positive
Name: Class, Length: 192, dtype: object

In [172]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df.head()


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,0
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,0
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,0
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,0
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,0


In [173]:
minor=df[df["Class"]==1]
major=df[df["Class"]==0]
minor=minor.drop(["Class"], axis=1)
minority=minor.to_numpy()
minor.shape
major.shape
diff=major.shape[0]-minor.shape[0]
r=diff
loop=diff/minor.shape[0]
if(diff%minor.shape[0]==0):
    iter=int(diff/minor.shape[0])
else:
  iter=int(diff/minor.shape[0])+1
minor.shape


(17, 9)

In [174]:
for id in range(r):
  if(diff>0):
    distances = np.linalg.norm(minority - minority[id], axis=1)  #calculating the distances to other instances
    k = 4
    nearest_neighbor_ids = distances.argsort()[:k] #finding 3 nearest neighbours n1,n2 and n3
    n1=minority[nearest_neighbor_ids[1]]
    n2=minority[nearest_neighbor_ids[2]]
    n3=minority[nearest_neighbor_ids[3]]
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    w=random.uniform(0.5,1)
    velocity= w+r1*(n1-n3) + r2*(n1-n2)         #velocity vector
    new_minor=minority[id]+velocity             #position update
    d=minor.shape[1] 
    minB=minor.min().to_numpy()
    maxB=minor.max().to_numpy()
    for i in range(d):
      if(new_minor[i]<minB[i]):
        new_minor[i]=minB[i];
      if(new_minor[i]>maxB[i]):
        new_minor[i]=maxB[i];
    minority=np.append(minority,new_minor)
    minority=np.reshape(minority,(-1, minor.shape[1]))
    diff=diff-1

minority.shape

(175, 9)

In [175]:
df_minor = pd.DataFrame(minority, columns =['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe'])
df_minor

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.517756,13.52965,3.40791,1.51943,72.0388,0.57753,8.78712,0.0000,0.0000
1,1.516093,13.33015,3.52914,1.33967,72.6716,0.55890,8.33520,0.0000,0.0000
2,1.516640,13.13730,3.44832,1.76018,72.4812,0.60237,8.37824,0.0000,0.0867
3,1.521219,14.02840,3.75813,0.57890,71.7924,0.11178,9.64792,0.0000,0.0000
4,1.516458,13.03755,3.39893,1.25942,73.0076,0.52164,8.58268,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...
170,1.522107,14.32100,3.90181,1.76018,73.0076,0.60858,9.64792,0.1512,0.4590
171,1.522107,14.32100,3.90181,1.76018,73.0076,0.60858,9.64792,0.1512,0.4590
172,1.522107,14.32100,3.90181,1.76018,73.0076,0.60858,9.64792,0.1512,0.4590
173,1.522107,14.32100,3.90181,1.76018,73.0076,0.60858,9.64792,0.1512,0.4590


In [176]:
dup=[]
for i in range(df_minor.shape[0]):
  dup.append(1)
Class=np.array(dup)
Class=np.reshape(Class,(-1,1))
df_minor_class=pd.DataFrame(Class,columns=['Class'])
df_minor['Class']=df_minor_class
df_minor

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Class
0,1.517756,13.52965,3.40791,1.51943,72.0388,0.57753,8.78712,0.0000,0.0000,1
1,1.516093,13.33015,3.52914,1.33967,72.6716,0.55890,8.33520,0.0000,0.0000,1
2,1.516640,13.13730,3.44832,1.76018,72.4812,0.60237,8.37824,0.0000,0.0867,1
3,1.521219,14.02840,3.75813,0.57890,71.7924,0.11178,9.64792,0.0000,0.0000,1
4,1.516458,13.03755,3.39893,1.25942,73.0076,0.52164,8.58268,0.0000,0.0000,1
...,...,...,...,...,...,...,...,...,...,...
170,1.522107,14.32100,3.90181,1.76018,73.0076,0.60858,9.64792,0.1512,0.4590,1
171,1.522107,14.32100,3.90181,1.76018,73.0076,0.60858,9.64792,0.1512,0.4590,1
172,1.522107,14.32100,3.90181,1.76018,73.0076,0.60858,9.64792,0.1512,0.4590,1
173,1.522107,14.32100,3.90181,1.76018,73.0076,0.60858,9.64792,0.1512,0.4590,1


In [177]:
majority=major.to_numpy()
majority=np.reshape(majority,(-1,major.shape[1]))
majority
df_major = pd.DataFrame(majority, columns = ['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','Class'])
df_major

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.00000,0.1224,0.0
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.00000,0.0000,0.0
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.00000,0.0000,0.0
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.00000,0.0000,0.0
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.00000,0.0561,0.0
...,...,...,...,...,...,...,...,...,...,...
170,1.516572,14.79980,0.00000,1.99130,73.1084,0.00000,8.28140,1.71045,0.0000,0.0
171,1.517323,14.95275,0.00000,1.79870,72.9908,0.00000,8.61496,1.54980,0.0000,0.0
172,1.516139,13.88210,1.77804,1.78907,73.0972,0.00000,8.66876,0.75915,0.0000,0.0
173,1.515091,15.15225,0.00000,2.25131,73.5004,0.00000,8.33520,0.63000,0.0000,0.0


In [178]:
balanced_df=pd.concat([df_minor,df_major],axis=0,ignore_index=True)
balanced_df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Class
0,1.517756,13.52965,3.40791,1.51943,72.0388,0.57753,8.78712,0.00000,0.0000,1.0
1,1.516093,13.33015,3.52914,1.33967,72.6716,0.55890,8.33520,0.00000,0.0000,1.0
2,1.516640,13.13730,3.44832,1.76018,72.4812,0.60237,8.37824,0.00000,0.0867,1.0
3,1.521219,14.02840,3.75813,0.57890,71.7924,0.11178,9.64792,0.00000,0.0000,1.0
4,1.516458,13.03755,3.39893,1.25942,73.0076,0.52164,8.58268,0.00000,0.0000,1.0
...,...,...,...,...,...,...,...,...,...,...
345,1.516572,14.79980,0.00000,1.99130,73.1084,0.00000,8.28140,1.71045,0.0000,0.0
346,1.517323,14.95275,0.00000,1.79870,72.9908,0.00000,8.61496,1.54980,0.0000,0.0
347,1.516139,13.88210,1.77804,1.78907,73.0972,0.00000,8.66876,0.75915,0.0000,0.0
348,1.515091,15.15225,0.00000,2.25131,73.5004,0.00000,8.33520,0.63000,0.0000,0.0


In [179]:

from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
x_b=balanced_df.drop(['Class'],axis='columns')
y_b=balanced_df.Class




In [180]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
model_b_s=SVC()

model_b_k=KNeighborsClassifier(n_neighbors=3)


model_b_d=DecisionTreeClassifier(random_state=0)

In [181]:
#precision
precision_s=[]
f1_s=[]
recall_s=[]
accuracy_s=[]
Auc_s=[]
Gmean_s=[]

precision_k=[]
f1_k=[]
recall_k=[]
accuracy_k=[]
Auc_k=[]
Gmean_k=[]

precision_d=[]
f1_d=[]
recall_d=[]
accuracy_d=[]
Auc_d=[]
Gmean_d=[]
for train_index_b,test_index_b in kf.split(x_b,y_b):
  x_train_b,x_test_b=x_b.iloc[train_index_b],x_b.iloc[test_index_b]
  y_train_b,y_test_b=y_b.iloc[train_index_b],y_b.iloc[test_index_b]
  model_b_s.fit(x_train_b,y_train_b)
  prediction_b_s = model_b_s.predict(x_test_b)

  model_b_k.fit(x_train_b,y_train_b)
  prediction_b_k = model_b_k.predict(x_test_b)

  model_b_d.fit(x_train_b,y_train_b)
  prediction_b_d = model_b_d.predict(x_test_b)

  precision_s.append(precision_score(y_test_b,prediction_b_s,average='weighted',labels=np.unique(prediction_b_s)))
  precision_k.append(precision_score(y_test_b,prediction_b_k,average='weighted',labels=np.unique(prediction_b_k)))
  precision_d.append(precision_score(y_test_b,prediction_b_d,average='weighted',labels=np.unique(prediction_b_d)))

  recall_s.append(recall_score(y_test_b,prediction_b_s,average='weighted',labels=np.unique(prediction_b_s)))
  recall_k.append(recall_score(y_test_b,prediction_b_k,average='weighted',labels=np.unique(prediction_b_k)))
  recall_d.append(recall_score(y_test_b,prediction_b_d,average='weighted',labels=np.unique(prediction_b_d)))

  f1_s.append(f1_score(y_test_b,prediction_b_s,average='weighted',labels=np.unique(prediction_b_s)))
  f1_k.append(f1_score(y_test_b,prediction_b_k,average='weighted',labels=np.unique(prediction_b_k)))
  f1_d.append(f1_score(y_test_b,prediction_b_d,average='weighted',labels=np.unique(prediction_b_d)))

  accuracy_s.append(accuracy_score(y_test_b,prediction_b_s))
  accuracy_k.append(accuracy_score(y_test_b,prediction_b_k))
  accuracy_d.append(accuracy_score(y_test_b,prediction_b_d))


  s_fpr_b,s_tpr_b,threshold_s_b=roc_curve(y_test_b,prediction_b_s)
  k_fpr_b,k_tpr_b,threshold_k_b=roc_curve(y_test_b,prediction_b_k)
  d_fpr_b,d_tpr_b,threshold_d_b=roc_curve(y_test_b,prediction_b_d)


  Auc_s.append(auc(s_fpr_b,s_tpr_b))
  Auc_k.append(auc(k_fpr_b,k_tpr_b))
  Auc_d.append(auc(d_fpr_b,d_tpr_b))


  Gmean_s.append(gmean(y_test_b,prediction_b_s))
  Gmean_k.append(gmean(y_test_b,prediction_b_k))
  Gmean_d.append(gmean(y_test_b,prediction_b_d))

print("-------SVM--------")
print("Gmean_SVM:",Gmean_s)
print("AUC_SVM:",Auc_s)
print("f1_SVM:",f1_s)
print("precision_SVM:",precision_s)
print("recall_SVM:",recall_s)

print("mean of Gmean_s:",stat.mean(Gmean_s))
print("mean of AUC_s:",stat.mean(Auc_s))
print("mean of f1_s:",stat.mean(f1_s))
print("mean of precision_s:",stat.mean(precision_s))
print("mean of recall_s:",stat.mean(recall_s))

print("standard deviation of Gmean_s:",stat.stdev(Gmean_s))
print("standard deviation of AUC_s:",stat.stdev(Auc_s))
print("standard deviation of f1_s:",stat.stdev(f1_s))
print("standard deviation of precision_s:",stat.stdev(precision_s))
print("standard deviation of recall_s:",stat.stdev(recall_s))

print("\n")
print("-------KNN--------")
print("Gmean_KNN:",Gmean_k)
print("AUC_KNN:",Auc_k)
print("f1_KNN:",f1_k)
print("precision_KNN:",precision_k)
print("recall_KNN:",recall_k)

print("mean of Gmean_k:",stat.mean(Gmean_k))
print("mean of AUC_k:",stat.mean(Auc_k))
print("mean of f1_k:",stat.mean(f1_k))
print("mean of precision_k:",stat.mean(precision_k))
print("mean of recall_k:",stat.mean(recall_k))

print("standard deviation of Gmean_k:",stat.stdev(Gmean_k))
print("standard deviation of AUC_k:",stat.stdev(Auc_k))
print("standard deviation of f1_k:",stat.stdev(f1_k))
print("standard deviation of precision_k:",stat.stdev(precision_k))
print("standard deviation of recall_k:",stat.stdev(recall_k))

print("\n")
print("-------DECISION TREE--------")
print("Gmean_DT:",Gmean_d)
print("AUC_DT:",Auc_d)
print("f1_DT:",f1_d)
print("precision_DT:",precision_d)
print("recall_DT:",recall_d)

print("mean of Gmean_d:",stat.mean(Gmean_d))
print("mean of AUC_d:",stat.mean(Auc_d))
print("mean of f1_d:",stat.mean(f1_d))
print("mean of precision_d:",stat.mean(precision_d))
print("mean of recall_d:",stat.mean(recall_d))

print("standard deviation of Gmean_d:",stat.stdev(Gmean_d))
print("standard deviation of AUC_d:",stat.stdev(Auc_d))
print("standard deviation of f1_d:",stat.stdev(f1_d))
print("standard deviation of precision_d:",stat.stdev(precision_d))
print("standard deviation of recall_d:",stat.stdev(recall_d))



-------SVM--------
Gmean_SVM: [0.701964118163034, 0.701964118163034, 0.6918633884230078, 0.6825283070704619, 0.8411529883206652]
AUC_SVM: [0.5, 0.5, 0.5142857142857142, 0.5285714285714286, 0.9142857142857143]
f1_SVM: [0.6666666666666666, 0.6666666666666666, 0.36431623931623935, 0.3938598792967724, 0.9136513157894737]
precision_SVM: [0.5, 0.5, 0.7536231884057971, 0.7573529411764706, 0.9268292682926829]
recall_SVM: [1.0, 1.0, 0.5142857142857142, 0.5285714285714286, 0.9142857142857143]
mean of Gmean_s: 0.7238945840280405
mean of AUC_s: 0.5914285714285714
mean of f1_s: 0.6010321535471638
mean of precision_s: 0.6875610795749901
mean of recall_s: 0.7914285714285714
standard deviation of Gmean_s: 0.06604655631557188
standard deviation of AUC_s: 0.18087090897733032
standard deviation of f1_s: 0.2265509294254902
standard deviation of precision_s: 0.1849612725977323
standard deviation of recall_s: 0.24899799195977465


-------KNN--------
Gmean_KNN: [0.6663381785133576, 0.8186103862277911, 1.0, 0

In [184]:
list=['glass016vs2',stat.mean(Auc_k),stat.stdev(Auc_k),stat.mean(Gmean_k),stat.stdev(Gmean_k),stat.mean(Auc_d),stat.stdev(Auc_d),stat.mean(Gmean_d),stat.stdev(Gmean_d),stat.mean(Auc_s),stat.stdev(Auc_s),stat.mean(Gmean_s),stat.stdev(Gmean_s)]
list

df_excel = pd.read_excel('/content/drive/MyDrive/pso_1.xlsx')
df_excel


,Unnamed: 0.1,KNN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Decision Tree,Unnamed: 6,Unnamed: 7,Unnamed: 8,SVM,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN
1,Dataset,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev
2,ecoli0137vs26,0.959495,0.040136,0.92449,0.072456,0.959798,0.027859,0.923228,0.052104,0.946633,0.061364,0.905228,0.105836
3,shuttle0vs4,0.999414,0.000802,0.998826,0.001607,1.0,0.0,1.0,0.0,0.998828,0.000655,0.997651,0.001313
4,yeast1vs7,0.953475,0.075796,0.923035,0.116982,0.938317,0.063328,0.89309,0.093762,0.963953,0.080602,0.94367,0.125957
5,shuttle2vs4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [185]:
df_excel.loc[len(df_excel)]=list
df_excel.to_excel('/content/drive/MyDrive/pso_1.xlsx')